<a href="https://colab.research.google.com/github/humayun-mhk/Elevvo-NLP-Internship/blob/main/Resume_Screening_Using_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install Required Libraries

In [1]:
!pip install pandas scikit-learn sentence-transformers python-docx pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 340.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3

## 2. Load Pretrained Sentence Embedding Model

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast and good for semantic similarity


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## 3. Example: Resume vs Job Description

In [3]:
resume_text = """
Experienced data analyst with 3+ years working in Python, SQL, and Excel.
Skilled in data visualization (Power BI, Tableau), and statistical modeling.
Worked on customer segmentation and marketing analytics.
"""

job_description = """
Looking for a junior data analyst with experience in SQL, Excel, and Python.
Familiarity with data visualization tools like Tableau and knowledge of statistics required.
"""

resume_embedding = model.encode(resume_text)
job_embedding = model.encode(job_description)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## 4. Compute Cosine Similarity

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([resume_embedding], [job_embedding])[0][0]
match_score = round(similarity * 100, 2)
print(f"Match Score: {match_score}%")


Match Score: 75.86000061035156%


## 5. Named Entity Recognition (Skills, Experience)

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(resume_text)
skills = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PRODUCT', 'SKILL']]
print("Extracted Skills/Entities:", skills)


Extracted Skills/Entities: ['SQL', 'Excel', 'Power BI']


## Upload Resume & Match with Job

In [6]:
import gradio as gr

def match_resume(resume_input, job_input):
    resume_embedding = model.encode(resume_input)
    job_embedding = model.encode(job_input)
    score = cosine_similarity([resume_embedding], [job_embedding])[0][0]
    return f"Match Score: {round(score * 100, 2)}%"

gr.Interface(
    fn=match_resume,
    inputs=[
        gr.Textbox(label="Paste Resume Text", lines=10),
        gr.Textbox(label="Paste Job Description", lines=5)
    ],
    outputs="text",
    title="Resume Matcher"
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://15639edbe9c4dcf6b2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Read Resume from File

In [8]:
file_path = "/content/Profile.pdf"

In [10]:
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        return "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
data = extract_text_from_pdf(file_path)


In [11]:
data

"Contact Humayun Mhk\nhumayunkhann47@gmail.com\nBuilding & Learning: Generative AI | ML | DL | Python\nwww.linkedin.com/in/humayun-\nMardān, Khyber Pakhtunkhwa, Pakistan\nmhk-927407251 (LinkedIn)\nExperience\nTop Skills\nMachine Learning Algorithms DevelopersHub Corporation©\nGenerative AI AI & Machine Learning Intern\nDeep Learning March 2025 - April 2025 (2 months)\nSuccessfully completed a six-week virtual internship focused on Artificial\nCertifications\nIntelligence and Machine Learning. Gained hands-on experience with ML\nSupervised Machine Learning: algorithms, model training, and deployment. Recognized for outstanding\nRegression and Classification\nperformance and contribution\nMachine Learning Specialization\nGenerative AI\nAdvanced Learning Algorithms\nEducation\nUnsupervised Learning,\nRecommenders, Reinforcement\nUniversity of Engineering & Technology Mardan\nLearning\nBachelor's degree, DATA SCIENCE · (September 2023 - June 2027)\nPage 1 of 1"